# Отчет Градиентный бустинг
### 1) Пропуски в данных.
#### Из вывода кода ниже видно в колонках с именами есть пропуски, ниже возможные причины, почему данные оказались с пропусками:
        first_blood_time - первой крови не было,
        first_blood_team - аналогично,
        first_blood_player1 - аналогично,
        first_blood_player2 - аналогично,
        radiant_bottle_time - боттл никто не покупал,
        radiant_courier_time - не купили курьера (хотя выборка, вроде, из про матчей, а не пабчика),
        radiant_flying_courier_time - аналогично - или не апнули курьера, или не купили,
        radiant_first_ward_time - не купили или не поставили варды,
        dire_bottle_time - аналогино,
        dire_courier_time - аналогично,
        dire_flying_courier_time - аналогично,
        dire_first_ward_time - аналогично.
### 2) Целевая переменная в столбце 'radiant_win'
### 3) Времени затрачено в зависимости от числа деревьев и полученное качество
        Trees number:  10 ROC-AUC average:  0.6643877206345741 Time spended:  0:00:33.209223
        Trees number:  20 ROC-AUC average:  0.6828535735340823 Time spended:  0:01:05.854935
        Trees number:  30 ROC-AUC average:  0.6894962060591201 Time spended:  0:01:40.367007
        Trees number:  40 ROC-AUC average:  0.6941311214730338 Time spended:  0:02:12.439259
        Trees number:  50 ROC-AUC average:  0.6974548316948366 Time spended:  0:02:42.516568
        
        Для ускорения можно увеличить параметр learning_rate.
### 4) Смысла нет, т.к значение ROC-AUC метрики практически не изменяется при дальнейшем увеличении числа деревьев. 

In [8]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import time
import datetime

### Считывание признаков и формирование обучающей выборки и массива с целевой переменной

In [160]:
features = pd.read_csv('./features.csv', index_col='match_id')
y = features['radiant_win']
features.drop(['duration',
        'radiant_win',
        'tower_status_radiant',
        'tower_status_dire',
        'barracks_status_dire',
        'barracks_status_radiant'], axis='columns', inplace=True)
X = features
#print (y)
#print (X)
test = pd.read_csv('./features_test.csv', index_col='match_id')

### Поиск и заполнение пропусков

In [161]:
test = test.fillna(0)
for col in X.columns:
    if (X[col].isnull().sum() > 0):
        print (col)
        X[col] = X[col].fillna(0)

first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


### Обучение классификаторов

In [49]:
cv = KFold(5, shuffle=True, random_state=241)
scores = []
times = []
for i in [10, 20, 30, 40, 50]:
    start_time = datetime.datetime.now()
    gbc = GradientBoostingClassifier(n_estimators=i);
    score = cross_val_score(X=X, y=y, estimator=gbc, cv=cv, scoring='roc_auc')
    scores.append(np.average(score))
    times.append(datetime.datetime.now() - start_time)

### Вывод итогов

In [50]:
for i in range(1, 6):
    print ("Trees number: ", 10 * i, "ROC-AUC average: ", scores[i - 1], "Time spended: ", times[i - 1])

Trees number:  10 ROC-AUC average:  0.6643877206345741 Time spended:  0:00:33.041424
Trees number:  20 ROC-AUC average:  0.6828535735340823 Time spended:  0:01:05.796829
Trees number:  30 ROC-AUC average:  0.6894962060591201 Time spended:  0:01:38.849133
Trees number:  40 ROC-AUC average:  0.6941311214730338 Time spended:  0:02:16.849018
Trees number:  50 ROC-AUC average:  0.6974548316948366 Time spended:  0:02:43.153568


# Отчет Логарифмическая регрессия
### 1) Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?
### 2) Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?
### 3) Сколько различных идентификаторов героев существует в данной игре?
### 4) Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?
### 5) Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

In [151]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import itertools

### Приведение признаков к одному масштабу

In [162]:
scaler = StandardScaler()
print (X.iloc[1])
X[X.columns] = scaler.fit_transform(X[X.columns])
print ("--------------")
print (X.iloc[1])

start_time                  1.430220e+09
lobby_type                  0.000000e+00
r1_hero                     4.200000e+01
r1_level                    4.000000e+00
r1_xp                       1.188000e+03
                                ...     
dire_tpscroll_count         5.000000e+00
dire_boots_count            4.000000e+00
dire_ward_observer_count    3.000000e+00
dire_ward_sentry_count      1.000000e+00
dire_first_ward_time       -5.000000e+00
Name: 1, Length: 102, dtype: float64
--------------
start_time                 -2.540452
lobby_type                 -0.927798
r1_hero                    -0.292258
r1_level                    0.501314
r1_xp                      -0.080139
                              ...   
dire_tpscroll_count         1.066668
dire_boots_count            0.562864
dire_ward_observer_count    0.678170
dire_ward_sentry_count      0.437788
dire_first_ward_time        0.043947
Name: 1, Length: 102, dtype: float64


### Обучение классификатора

In [146]:
cv_log = KFold(5, shuffle=True, random_state=241)
scores_log = []
times_log = []
for C in [0.1, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0]:
    start_time = datetime.datetime.now()
    lr = LogisticRegression(penalty='l2', C=C);
    score_log = cross_val_score(X=X, y=y, estimator=lr, cv=cv_log, scoring='roc_auc')
    scores_log.append(np.average(score_log))
    times_log.append(datetime.datetime.now() - start_time)

### Вывод результатов

In [157]:
C_values = [0.1, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0]
for i in range(11):
    print ("C:", C_values[i], "time:", times_log[i], "ROC-AUC:", scores_log[i])

C: 0.1 time: 0:00:02.977236 ROC-AUC: 0.7163104821911903
C: 0.2 time: 0:00:02.627419 ROC-AUC: 0.7163080584810423
C: 0.4 time: 0:00:02.608809 ROC-AUC: 0.7163068844675312
C: 0.6 time: 0:00:02.818090 ROC-AUC: 0.7163068291433263
C: 0.8 time: 0:00:02.598185 ROC-AUC: 0.7163071515611381
C: 1.0 time: 0:00:02.732703 ROC-AUC: 0.7163066323165939
C: 1.2 time: 0:00:02.615043 ROC-AUC: 0.716307085854079
C: 1.4 time: 0:00:02.620396 ROC-AUC: 0.7163063717278819
C: 1.6 time: 0:00:02.808783 ROC-AUC: 0.7163064097806202
C: 1.8 time: 0:00:02.613228 ROC-AUC: 0.7163064330833286
C: 2.0 time: 0:00:02.705744 ROC-AUC: 0.716306708587007


### Уберем категориальные признаки

In [163]:
X.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
       'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis='columns', inplace=True)

### Переобучаем

In [164]:
cv_log = KFold(5, shuffle=True, random_state=241)
scores_log = []
times_log = []
for C in [0.1, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0]:
    start_time = datetime.datetime.now()
    lr = LogisticRegression(penalty='l2', C=C);
    score_log = cross_val_score(X=X, y=y, estimator=lr, cv=cv_log, scoring='roc_auc')
    scores_log.append(np.average(score_log))
    times_log.append(datetime.datetime.now() - start_time)

### Без категориальных признаков

In [165]:
C_values = [0.1, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0]
for i in range(11):
    print ("C:", C_values[i], "time:", times_log[i], "ROC-AUC:", scores_log[i])

C: 0.1 time: 0:00:02.920308 ROC-AUC: 0.7163745519438582
C: 0.2 time: 0:00:02.707141 ROC-AUC: 0.7163720623579352
C: 0.4 time: 0:00:02.804573 ROC-AUC: 0.7163717888784846
C: 0.6 time: 0:00:02.818490 ROC-AUC: 0.7163711193089657
C: 0.8 time: 0:00:02.654374 ROC-AUC: 0.7163712656768272
C: 1.0 time: 0:00:02.765744 ROC-AUC: 0.7163709541660922
C: 1.2 time: 0:00:02.676005 ROC-AUC: 0.716370833454984
C: 1.4 time: 0:00:02.771467 ROC-AUC: 0.7163709245254307
C: 1.6 time: 0:00:02.910847 ROC-AUC: 0.7163710241640151
C: 1.8 time: 0:00:02.655922 ROC-AUC: 0.7163710262844653
C: 2.0 time: 0:00:02.726782 ROC-AUC: 0.7163710411455433


### Подсчет уникальных персонажей

#### Вернем данные

In [257]:
features = pd.read_csv('./features.csv', index_col='match_id')
y = features['radiant_win']
features.drop(['duration',
        'radiant_win',
        'tower_status_radiant',
        'tower_status_dire',
        'barracks_status_dire',
        'barracks_status_radiant'], axis='columns', inplace=True)
X = features
#print (y)
#print (X)
test = pd.read_csv('./features_test.csv', index_col='match_id')

In [208]:
heroes = X[['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
       'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']]
all_heroes = []
for col in heroes.columns:
    for x in heroes[col].values:
        all_heroes.append(x)

print (len(np.unique(all_heroes)))
print (max(all_heroes))

108
112


### Формирование мешка слов

In [251]:
# N — количество различных героев в выборке
N = 113
X_pick = np.zeros((X.shape[0], 113))


for i, match_id in enumerate(X.index):
    for p in range(5):
        X_pick[i][int(X.iloc[i]['r%d_hero' % (p+1)])] = 1
        X_pick[i][int(X.iloc[i]['d%d_hero' % (p+1)])] = -1

print (X_pick)

[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  1.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0. -1.]]


### Приведем к одному масштабу, заменим nan-значения, и заменим категориальные мешком слов

In [258]:
scaler = StandardScaler()
X[X.columns] = scaler.fit_transform(X[X.columns])
X.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
        'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis='columns', inplace=True)

In [259]:
pick_df = pd.DataFrame(X_pick, index=X.index)
X = X.join(pick_df)
print (X)

          start_time  r1_level     r1_xp   r1_gold     r1_lh  r1_kills  \
match_id                                                                 
0          -2.544364  1.400808  1.525972  0.734957  0.969743 -0.537757   
1          -2.540452  0.501314 -0.080139 -0.247570 -0.246859 -0.537757   
2          -2.539231  0.501314  0.151070  0.263085  1.190944 -0.537757   
3          -2.532622  0.501314  0.962950 -0.198013  0.306142 -0.537757   
4          -2.529221  0.501314  0.348745 -0.124754 -0.357459  0.968527   
...              ...       ...       ...       ...       ...       ...   
114402      1.093978  0.501314  0.834109  0.107949  0.637942 -0.537757   
114403      1.096181  0.501314  0.987660  0.577666  0.637942 -0.537757   
114404      1.098746  0.501314  0.292267 -1.309820 -1.131661 -0.537757   
114405      1.098952 -0.398181 -0.173682 -0.822866 -0.578660 -0.537757   
114406      1.102648 -0.398181 -0.318409 -0.751762 -0.468060 -0.537757   

          r1_deaths  r1_items  r2_lev

In [260]:
test = test.fillna(0)
for col in X.columns:
    if (X[col].isnull().sum() > 0):
        print (col)
        X[col] = X[col].fillna(0)

first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


###  Учим с мешком слов

In [265]:
cv_log = KFold(5, shuffle=True, random_state=241)
scores_log = []
times_log = []
for C in [0.1, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0]:
    start_time = datetime.datetime.now()
    lr = LogisticRegression(penalty='l2', C=C, max_iter=1000);
    score_log = cross_val_score(X=X, y=y, estimator=lr, cv=cv_log, scoring='roc_auc')
    scores_log.append(np.average(score_log))
    times_log.append(datetime.datetime.now() - start_time)

In [266]:
C_values = [0.1, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0]
for i in range(11):
    print ("C:", C_values[i], "time:", times_log[i], "ROC-AUC:", scores_log[i])

C: 0.1 time: 0:00:11.238751 ROC-AUC: 0.752036314611976
C: 0.2 time: 0:00:10.334547 ROC-AUC: 0.7520266250984584
C: 0.4 time: 0:00:10.070942 ROC-AUC: 0.7520211114867772
C: 0.6 time: 0:00:11.743307 ROC-AUC: 0.7520191371469503
C: 0.8 time: 0:00:09.650042 ROC-AUC: 0.7520199450729723
C: 1.0 time: 0:00:09.048122 ROC-AUC: 0.7520179210985909
C: 1.2 time: 0:00:11.057455 ROC-AUC: 0.7520185540245964
C: 1.4 time: 0:00:09.649043 ROC-AUC: 0.7520173443546087
C: 1.6 time: 0:00:10.318924 ROC-AUC: 0.7520166835623222
C: 1.8 time: 0:00:09.842797 ROC-AUC: 0.7520162130938314
C: 2.0 time: 0:00:10.394138 ROC-AUC: 0.7520156431223286


### Обучим с С = 0.1 последний классификатор и найдем предсказания на тестовой выборке

In [281]:
lr = LogisticRegression(penalty='l2', C=0.1, max_iter=1000);
lr.fit(X=X, y=y)

LogisticRegression(C=0.1, max_iter=1000)

In [272]:
test = pd.read_csv('./features_test.csv', index_col='match_id')
#test = scaler.transform(test)
#print(test)

In [274]:
test = test.fillna(0)
test[test.columns] = scaler.transform(test[test.columns])
#print (test)

In [275]:
# N — количество различных героев в выборке
N = 113
test_pick = np.zeros((test.shape[0], 113))


for i, match_id in enumerate(test.index):
    for p in range(5):
        test_pick[i][int(test.iloc[i]['r%d_hero' % (p+1)])] = 1
        test_pick[i][int(test.iloc[i]['d%d_hero' % (p+1)])] = -1

print (test_pick)

[[ 0.  0.  0. ...  0.  0. -1.]
 [ 0.  0.  0. ...  0.  0. -1.]
 [ 0.  0.  0. ...  0.  0. -1.]
 ...
 [ 0.  0.  0. ...  0.  0. -1.]
 [ 0.  0.  0. ...  0.  0. -1.]
 [ 0.  0.  0. ...  0.  0. -1.]]


In [276]:
test_pick_df = pd.DataFrame(test_pick, index=test.index)
test = test.join(test_pick_df)
print (test)

          start_time  lobby_type   r1_hero  r1_level     r1_xp   r1_gold  \
match_id                                                                   
6        -261.856116   -1.254977 -1.542905 -2.645736 -2.177315 -2.473613   
7        -261.856116   -1.130621 -1.611746 -4.263917 -2.179019 -2.476022   
10       -261.856116   -1.130621 -1.524987 -4.263917 -2.178411 -2.474917   
13       -261.856116   -1.130621 -1.605144 -3.454827 -2.178545 -2.474476   
16       -261.856116   -1.130621 -1.593828 -2.645736 -2.176829 -2.475600   
...              ...         ...       ...       ...       ...       ...   
114369   -261.856115   -0.384488 -1.620233 -1.836646 -2.174352 -2.469657   
114377   -261.856115   -1.130621 -1.627777 -3.454827 -2.178421 -2.475860   
114378   -261.856115   -1.130621 -1.550449 -4.263917 -2.178959 -2.476352   
114393   -261.856115   -1.254977 -1.624005 -2.645736 -2.175006 -2.473209   
114398   -261.856115   -1.130621 -1.623062 -2.645736 -2.176966 -2.472610   

           

In [278]:
test.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
        'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis='columns', inplace=True)

In [282]:
print (max(lr.predict_proba(test)[:, 1]), min(lr.predict_proba(test)[:, 1]))

0.0024095712106631394 0.00029156770779229825
